In [47]:
from gensim.test.utils import datapath
from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
import cupy as cp
import spacy
import cudf
import numpy as np
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [138]:
data_tip=pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True,)[5000]


KeyError: 5000

In [ ]:
tip_text = data_tip['text']

In [ ]:
tip_text[:3]

In [ ]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in doc_spacy]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

tip_text_preprocessed = tip_text.apply(process_text)



In [139]:
tip_text_preprocessed

0                                 [avenger, time, lady]
1                [good, desert, tasty, cuban, sandwich]
2                                         [open, think]
3                              [decent, fried, chicken]
4                  [appetizer, platter, special, lunch]
5     [chili, single, cheeseburger, onion, pickle, r...
6     [saturday, 2013, ride, patco, silver, sleigh, ...
7     [probably, good, place, cool, spring, area, wa...
8                                               [tacos]
9       [starbuck, substitute, boring, downtown, tampa]
10                              [order, tortilla, soup]
11                             [good, definitely, come]
12                                     [hotlight, stop]
13                                             [yankee]
14                             [basically, food, rally]
15    [dinner, close, yvonne, negative, star, restau...
16                                [right, right, tight]
17    [grill, chicken, entree, delicious, small,

In [140]:
model3= FastText(vector_size=300,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)

In [141]:
model3.wv.vectors_vocab 

array([[-0.13392863,  0.3249501 ,  0.0299471 , ...,  0.03083492,
         0.09707911,  0.23898022],
       [-0.08206087,  0.18223828, -0.25381836, ..., -0.3011456 ,
         0.129707  ,  0.01316063],
       [ 0.03069151,  0.38463068, -0.10569379, ..., -0.02991502,
        -0.03792377, -0.0118044 ],
       ...,
       [-0.11466099,  0.24085486, -0.18489273, ...,  0.10110843,
        -0.15032984,  0.38449937],
       [-0.15038449,  0.34601805, -0.21251695, ...,  0.09558389,
        -0.12009461,  0.4501465 ],
       [ 0.01581885,  0.09955907, -0.27963674, ..., -0.2077912 ,
         0.05239638,  0.0139199 ]], dtype=float32)

In [142]:
model3.ns_exponent #PROBABILITY OF NEGATIVE WORD SAMPLING

0.75

In [166]:
test_words="Pizza Burger Coffee Restaurant Delivery Review Menu Rating Service Ambience Reservation Cuisine Dessert Vegetarian Location Price Quality Fast Healthy Local Taste Atmosphere Barbecue Seafood Vegan Brunch Sushi Wine Outdoor Gourmet".split()
test_words=list(np.array(test_words).astype(str))

In [171]:
def test_words_(model, test_words):
    similar_words = []
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        similar_words.append(similar_word)
    return similar_words
similar_words = np.array(test_words_(model3, test_words))
print(f'the shape of outputs is {similar_words.shape}')
similar_words=list(similar_words)

the shape of outputs is (30, 10, 2)


In [ ]:
len(model3.wv.key_to_index)

In [ ]:
sims = model3.wv.most_similar('lots',topn=10) 

In [ ]:
sims